In [ ]:
import numpy as np
import cv2
import rosbag
from cv_bridge import CvBridge
from ros_numpy.point_cloud2 import pointcloud2_to_xyz_array
import pcl
from tf.transformations import quaternion_matrix
import cv2

# Standard plotly imports
import plotly.graph_objects as go
import plotly.express as px
import plotly.io as pio
pio.templates.default = 'plotly_dark'

In [ ]:
# Load rosbag containing depth map, point cloud and camera image
bag = rosbag.Bag("/home/user/rosbags/pcl_rampe.bag")

for topic, msg, t in bag.read_messages(topics="/zed2i/zed_node/left/image_rect_color"):
    # Convert from image msg to numpy array
    img = CvBridge().imgmsg_to_cv2(msg, desired_encoding="rgb8")
    break
for topic, msg, t in bag.read_messages(topics="/zed2i/zed_node/depth/depth_registered"):
    # Convert depth map to numpy array
    depth_img = CvBridge().imgmsg_to_cv2(msg, desired_encoding="32FC1")
    break
for topic, msg, t in bag.read_messages(topics="/zed2i/zed_node/point_cloud/cloud_registered"):
    # Convert from image msg to numpy array
    pc_array = pointcloud2_to_xyz_array(msg)
    break
bag.close()

Some functions

In [ ]:
def reduce_pc(pc, x_range, y_range, z_range):
    """Removes points outside of box"""
    # Filter array
    pc_cut = pc[
        (pc[:, 0] > x_range[0])
        & (pc[:, 0] < x_range[1])
        & (pc[:, 1] > y_range[0])
        & (pc[:, 1] < y_range[1])
        & (pc[:, 2] > z_range[0])
        & (pc[:, 2] < z_range[1])
    ]
    return pc_cut

def reduce_pc2d(pc, x_range, y_range):
    """Removes points outside of box"""
    # Indices of points inside box
    inliers = [
        (pc[:, 0] > x_range[0])
        & (pc[:, 0] < x_range[1])
        & (pc[:, 1] > y_range[0])
        & (pc[:, 1] < y_range[1])
    ][0]
    # Filter array
    pc_cut = pc[inliers]
    return pc_cut, inliers


def voxel_filter(pc, leaf_size):
    """Downsample point cloud using voxel filter"""
    vgf = pc.make_voxel_grid_filter()
    # Leaf_size is the length of the side of the voxel cube in m
    vgf.set_leaf_size(leaf_size, leaf_size, leaf_size)
    pc_filtered = vgf.filter()
    return pc_filtered

def array_to_pcl(pc_array):
    """Gets pcl point cloud from numpy array"""
    pc = pcl.PointCloud()
    pc.from_array(pc_array.astype("float32"))
    return pc

def ransac(pc):
    """Find inliers and normal vector of dominant plane"""
    # 50?
    seg = pc.make_segmenter_normals(50)
    # Doubles the speed if True
    seg.set_optimize_coefficients(True)
    seg.set_model_type(pcl.SACMODEL_NORMAL_PLANE)
    seg.set_method_type(pcl.SAC_RANSAC)
    # How close a point must be to model to be considered inlier
    seg.set_distance_threshold(0.11)
    # normal_distance_weight?
    seg.set_normal_distance_weight(0.01)
    # How many tries
    seg.set_max_iterations(100)
    inliers_idx, coefficients = seg.segment()
    return inliers_idx, coefficients

def project_pointcloud(pc):
    # Transformation from pointcloud (camera left) to left camera lens (optical) (using output from tf node)
    trans = [0.0, 0.0, 0.01]
    quat = [0.5, -0.49999999999755174, 0.5, 0.5000000000024483]

    # Pointcloud points
    objectPoints = np.asarray(pc, dtype=np.float64)

    # Get rotation vector from lidar frame to camera frame
    # Convert quaternion to rotation matrix
    rotMat = quaternion_matrix(quat)[:3, :3]
    # Convert rotation matrix to rotation vector
    rvec, _ = cv2.Rodrigues(rotMat)

    # Translation vector from lidar to camera frame
    tvec = np.asarray(trans)

    # Camera matrix (K)
    cameraMatrix = np.asarray(
        [539.15087890625, 0.0, 564.685546875, 0.0, 539.15087890625, 316.5738220214844, 0.0, 0.0, 1.0]
    )
    cameraMatrix = np.reshape(cameraMatrix, (3, 3))


    # Distortion coefficients (D)
    distCoeffs = np.array([0.0, 0.0, 0.0, 0.0, 0.0])

    # Projection of 3D points onto 2D plane
    lidar_img_points, _ = cv2.projectPoints(objectPoints, rvec, tvec, cameraMatrix, distCoeffs)
    lidar_img_points = lidar_img_points.reshape(len(lidar_img_points), 2)
    return lidar_img_points

### Visualize all three views

In [ ]:
# IMAGE
px.imshow(img).show()

# DEPTH MAP
# Replace points without range information
# depth_img_no_nans = np.nan_to_num(depth_img, 0)
px.imshow(depth_img).show()

# POINTCLOUD
# Convert point cloud from numpy to pcl format
pc = array_to_pcl(pc_array)
# Downsample point cloud
pc_small = voxel_filter(pc, 0.1)
pc_small = np.asarray(pc_small)
# Project point cloud to 2D image
pc_2d = project_pointcloud(pc_small)
fig = px.imshow(img)
fig.add_trace(go.Scatter(x=pc_2d[:, 0], y=pc_2d[:, 1], mode="markers", marker=dict(size=2)))

### Load the detected bounding box from detectron2

Using the trained network, detectron2 outputs the following box $(x_0,y_0,w,h)$

``[437, 399, 240, 121]``

This box can now be cut from the image to get the ramp region

In [ ]:
# Box dimensions
box = [437, 399, 240, 121]
# Get box coorinates
x_range = [box[0], box[0] + box[2]]
y_range = [box[1], box[1] + box[3]]

# Visualize bounding box in image
fig = px.imshow(img)
fig.add_shape(type="rect", x0=box[0], y0=box[1], x1=box[0]+box[2], y1=box[1]+box[3], line=dict(color="red", width=2))
fig.show()

# Crop to bounding box in the point cloud
# Filter points outside of bounding box
pc_ramp_2d, inliers = reduce_pc2d(pc_2d, x_range, y_range)
fig = px.imshow(img)
fig.add_trace(go.Scatter(x=pc_ramp_2d[:, 0], y=pc_ramp_2d[:, 1], mode="markers", marker=dict(size=2)))
fig.show()

# Crop to bounding box in the 2D image
# 2D image
crop_img = img.copy()
crop_img = crop_img[box[1] : box[1] + box[3], box[0] : box[0] + box[2]]
px.imshow(crop_img).show()

# Crop to bounding box in the depth map
crop_depth_img = depth_img.copy()
crop_depth_img = crop_depth_img[box[1] : box[1] + box[3], box[0] : box[0] + box[2]]
px.imshow(crop_depth_img).show()


### Apply RANSAC to extracted ramp Pointcloud

In [ ]:
# Extract ramp region from (downsampled) 3D point cloud
pc_ramp = pc_small[inliers]

# Apply RANSAC to find dominant plane
pc_ramp_indices, _ = ransac(array_to_pcl(pc_ramp))
pc_ramp_plane = np.asarray(array_to_pcl(pc_ramp).extract(pc_ramp_indices))

go.Figure(go.Scatter3d(x=pc_ramp[:, 0], y=pc_ramp[:, 1], z=pc_ramp[:, 2], mode='markers', marker=dict(size=1))).show()
go.Figure(go.Scatter3d(x=pc_ramp_plane[:, 0], y=pc_ramp_plane[:, 1], z=pc_ramp_plane[:, 2], mode='markers', marker=dict(size=1))).show()